In [3]:
import gower
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist as pdist
from scipy.spatial.distance import squareform as sf
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [5]:
# load the daataset and remove the last two columns
dataset = pd.read_csv('Data/dataset.csv', sep = ";", decimal=",")
dataset = dataset.iloc[:, 1:-2]

In [6]:
[N,M] = np.shape(dataset)  
PAL = ['green', 'blue', 'yellow', 'orange', 'purple', 'magenta', 'cyan', 'brown', 'black', 'red']

In [14]:
dist_matrix = gower.gower_matrix(dataset)

In [28]:
dataset.head()

,Dim_0,Dim_1=0,Dim_2=0,Dim_3=0,Dim_4=0,Dim_5=0,Dim_6=0,Dim_7=0,Dim_8=0,Dim_9=0,...,Dim_11=0,Dim_12=0,Dim_13=0,Dim_14=0,Dim_15=0,Dim_16,Dim_17,Dim_18,Dim_19,Dim_20
0,0.750000,1,0,1,1,1,1,1,0,1,...,1,1,1,1,1,0.001132,0.080780,0.197324,0.300926,0.225000
1,0.239583,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0.000472,0.164345,0.235786,0.537037,0.165625
2,0.479167,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0.003585,0.130919,0.167224,0.527778,0.118750
3,0.656250,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0.001698,0.091922,0.125418,0.337963,0.129688
4,0.229167,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0.000472,0.142061,0.229097,0.337963,0.235938


In [30]:
import pandas as pd

def pandas_to_typst(df):
    typst_text = "\n#table(\n"
    
    # Get number of columns
    num_columns = len(df.columns)
    typst_text += f"  columns: {num_columns},\n"
    
    # Get column names
    typst_text += "  "
    for col in df.columns:
        typst_text += f"[*{col}*], "
    typst_text = typst_text[:-2] + ",\n"
    
    # Get values
    for index, row in df.iterrows():
        typst_text += "  "
        for value in row:
            typst_text += f"[{value}], "
        typst_text = typst_text[:-2] + ",\n"
    
    typst_text += ") \n"
    
    return typst_text

# Example DataFrame
data = {
    'Amount': ['360g', '250g', '150g', '100g', '100g', '100g', '2', 'Pinch', 'Drizzle'],
    'Ingredient': ['Baking flour', 'Butter (room temp.)', 'Brown sugar', 'Cane sugar', 
                   '70% cocoa chocolate', '35-40% cocoa chocolate', 'Eggs', 'Salt', 'Vanilla extract']
}
df = pd.DataFrame(dataset.head())

typst_table = pandas_to_typst(df)
print(typst_table)


#table(
  columns: 21,
  [*Dim_0*], [*Dim_1=0*], [*Dim_2=0*], [*Dim_3=0*], [*Dim_4=0*], [*Dim_5=0*], [*Dim_6=0*], [*Dim_7=0*], [*Dim_8=0*], [*Dim_9=0*], [*Dim_10=0*], [*Dim_11=0*], [*Dim_12=0*], [*Dim_13=0*], [*Dim_14=0*], [*Dim_15=0*], [*Dim_16*], [*Dim_17*], [*Dim_18*], [*Dim_19*], [*Dim_20*],
  [0.75], [1.0], [0.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.001132], [0.08078], [0.197324], [0.300926], [0.225],
  [0.239583], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.000472], [0.164345], [0.235786], [0.537037], [0.165625],
  [0.479167], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.003585], [0.130919], [0.167224], [0.527778], [0.11875],
  [0.65625], [0.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.001698], [0.091922], [0.125418], [0.337963], [0.12968